In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import cv2
import numpy as np
from PIL import Image, ImageDraw
import tensorflow as tf
import time
import json
import os
from tqdm import tqdm

import sys
sys.path.append('/home/dan/work/cocoapi/PythonAPI/')

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('../inference/')
from detector import Detector

In [ ]:
PATH_TO_FROZEN_GRAPH = '../model.pb'

# Build a model

In [ ]:
detector = Detector(PATH_TO_FROZEN_GRAPH, gpu_memory_fraction=0.5, visible_device_list='0')

# Do inference

In [ ]:
def visualize(image_array, boxes, scores):
    
    image = Image.fromarray(image_array)
    draw = ImageDraw.Draw(image)
    
    for i in range(len(boxes)):
        
        ymin, xmin, ymax, xmax = boxes[i]
        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)], outline='red'
        )
        draw.text((xmin, ymin), str(scores[i]), fill='red')
    
    return image

In [ ]:
image_path = '/mnt/datasets/bba/train/images/0_108_2017_11_20_14:41:07.jpg'
image = cv2.imread(image_path)
# WIDTH, HEIGHT = 1920 // 3, 1080 // 3
# image = cv2.resize(image, (WIDTH, HEIGHT), cv2.INTER_NEAREST)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

times = []
for _ in range(510):
    start = time.perf_counter() 
    boxes, _, scores = detector(image)
    times.append(time.perf_counter() - start)
        
times = np.array(times[10:])
print(times.mean(), times.std())

In [ ]:
visualize(image, boxes, scores)

# Load COCO groundtruth

In [ ]:
IMAGES_DIR = '/home/gpu2/hdd/dan/COCO/images/'
ANNOTATIONS_DIR = '/home/gpu2/hdd/dan/COCO/annotations/'
DATA_TYPE = 'val2017'  # train2017 or val2017

In [ ]:
coco = COCO(os.path.join(ANNOTATIONS_DIR, 'instances_{}.json'.format(DATA_TYPE)))

# Compute predictions

In [ ]:
catIds = coco.getCatIds(catNms=['person'])
imgIds = coco.getImgIds(catIds=catIds)

In [ ]:
results = []

for image_id in tqdm(imgIds):

    image_metadata = coco.loadImgs(image_id)[0]
    image_path = os.path.join(IMAGES_DIR, DATA_TYPE, image_metadata['file_name'])
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, _ = image.shape

    boxes, _, scores = detector(image, score_threshold=0.01)

    for i in range(len(boxes)):
        ymin, xmin, ymax, xmax = boxes[i]

        x, y = int(xmin), int(ymin)
        w, h = int(xmax - xmin), int(ymax - ymin)

        results.append({
            "image_id" : int(image_metadata['id']), "category_id" : 1, 
            "bbox" : [x, y, w, h], "score" : float(scores[i])
        })

In [ ]:
with open('coco_predictions.json', 'w') as f:
    json.dump(results, f)

# Show detections

In [ ]:
coco_detections = coco.loadRes('coco_predictions.json')

In [ ]:
def get_random():
    i = np.random.randint(0, len(imgIds))
    image_id = imgIds[i]
    image_metadata = coco.loadImgs(image_id)[0]
    image_path = os.path.join(IMAGES_DIR, DATA_TYPE, image_metadata['file_name'])
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    annIds = coco_detections.getAnnIds(imgIds=image_metadata['id'], catIds=catIds)
    annotations = coco_detections.loadAnns(annIds)
    return image, annotations

In [ ]:
image, annotations = get_random()

plt.figure(dpi=100, figsize=(17, 10))
plt.imshow(image)
plt.axis('off')
coco.showAnns(annotations);

# Evaluate

In [ ]:
cocoEval = COCOeval(cocoGt=coco, cocoDt=coco_detections, iouType='bbox')
cocoEval.params.imgIds = imgIds
cocoEval.params.catIds = catIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()